In [49]:
import os
import pandas as pd
import re
import numpy as np
import glob
from stop_words import get_stop_words
from num2words import num2words 

In [50]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS/quip"
os.chdir(path)
# quip_file = os.listdir(path)
# quip_file=quip_file[1]
# quip_file=[f for f in os.listdir(path) if not f.startswith('.')]
# quip_df = pd.read_excel(quip_file[1])
quip_df = pd.read_excel('Q1_21.xlsx')

In [51]:
string_val_9='FY21W0'
string_val_above9='FY21W'
def week_number_9(no):
    if(int(no) <= 9):
        return (string_val_9 + no)
    else:
        return (string_val_above9 + no)

In [52]:
quip_df=quip_df[(quip_df['Status with TARS'] == 'Open')].copy()
quip_df['Weeks impacted']=quip_df['Weeks impacted'].astype(str)
quip_df['Weeks impacted'] = quip_df.apply(lambda x: x['Weeks impacted'].split(","), axis=1)

# for index,week_ls in enumerate(quip_df['Weeks impacted']):
#     quip_df['Weeks impacted'].iloc[index] = [week_number_9(x) for x in week_ls]
for index,week_ls_row in quip_df.iterrows():
    quip_df.at[index,'Weeks impacted']= [week_number_9(x) for x in week_ls_row['Weeks impacted']]

In [53]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS/GBI"
os.chdir(path)
GBI_df = pd.read_excel('Reloads_Q1_21.xlsx')
# print(GBI_df.head())
GBI_df.columns=GBI_df.iloc[0]
GBI_df.drop(GBI_df.index[[0]],inplace=True)

In [54]:
print('GBI Dimension',GBI_df.shape)
# print('GBI Columns',GBI_df.columns)

GBI Dimension (877015, 27)


In [55]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS"
os.chdir(path)
Admin_df = pd.read_excel('TARS Analysts - Q2FY21 Responsibilities & Partner Assignments.xlsx')
quip_admin_df = pd.merge(quip_df,Admin_df[['Partner TARS B2B Analyst','Apple HQ ID']],on='Apple HQ ID', how='left')

In [56]:
# GBI_df[(GBI_df['Apple HQ ID']=='2638065')]
# GBI_df[(GBI_df['Fiscal Week/Year (Name)']=='FY21W05') & (GBI_df['Apple HQ ID']==303923)]

In [57]:
path="/Users/simranpal20gmail.com/Documents/Apple/Data/RELOADS/Admins"
os.chdir(path)
for index, admin_row in quip_admin_df.iterrows():
    main_admin_df = pd.DataFrame(columns=GBI_df.columns)
    main_admin_df = GBI_df[(GBI_df['Fiscal Week/Year (Name)'].isin(admin_row['Weeks impacted'])) & (GBI_df['Apple HQ ID']==str(admin_row['Apple HQ ID']))]
    week_nums='-'.join(['W'+x[-2:] for x in admin_row['Weeks impacted']])
    excel_string=admin_row['Partner TARS B2B Analyst'].split(' ')[0]+' '+str(admin_row['Apple HQ ID'])+' '+week_nums+'.xlsx'
    w_write = pd.ExcelWriter(excel_string)
    main_admin_df.to_excel(w_write,index=False)
    w_write.save()